<img src="mioti.png" style="height: 100px">
<center style="color:#888">Módulo Data Science in IoT<br/>Asignatura Data preprocessing</center>

# Worksheet S2: Filtrado y Anomización

## Objetivos

El objetivo de este worksheet es que aprendas los métodos más comunes de anonimización y filtrado.

* Filtrado con máscaras
* Generación de datos ficticios
* Anonimización

## Configuración del entorno

Como siempre, es importante que definamos en un solo bloque los paquetes y constantes a utilizar.

In [2]:
# instalar requisitos
import sys
!{sys.executable} -m pip install faker

     |████████████████████████████████| 1.0MB 5.4MB/s eta 0:00:01
     |████████████████████████████████| 81kB 23.1MB/s eta 0:00:01


In [1]:
%matplotlib inline

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

## Carga de los datos

Esta vez va a ser fácil, vamos a importar los datos de un fichero csv, utilizaremos la función read_csv que nos proporciona la libreria de pandas para cargar una base de datos de usuarios en un dataframe que llamaremos `df`.

In [3]:
df = pd.read_csv('usuarios.csv')

Una vez cargados los datos debemos inspeccionarlos antes de empezar nuestro análisis.

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 10 columns):
Nombre           100 non-null object
Apellido 1       100 non-null object
Apellido 2       100 non-null object
Sexo             96 non-null object
Municipio        99 non-null object
Provincia        100 non-null object
DNI              100 non-null int64
NIF              99 non-null object
Edad             99 non-null float64
ColorFavorito    99 non-null object
dtypes: float64(1), int64(1), object(8)
memory usage: 7.9+ KB


In [5]:
df.head()

,Nombre,Apellido 1,Apellido 2,Sexo,Municipio,Provincia,DNI,NIF,Edad,ColorFavorito
0,Francisco,Castro,Cano,H,Lérida,Lérida,8805982,H,6.0,Verde
1,Xavier,Gómez,Rendón,H,Córdoba,Córdoba,26616576,X,77.0,Azul
2,Carmen,Vázquez,Trenado,M,Sabadell,Barcelona,47915145,NaN,NaN,Rojo
3,Anna,Alonso,López,M,NaN,Zaragoza,23362379,Z,24.0,Rojo
4,Manuel,López,Martínez,H,Palma de Mallorca,Islas Baleares,54999682,N,28.0,Amarillo


## Navegación en el dataframe

Antes de empezar a realizarnos preguntas o hacer transformaciones sobre los datos, debemos comprender estructuran tienen. Para ello podemos hacer uso de las funciones `head`, `index` y `columns` para identificar la estructura del dataset.

In [6]:
df.head() # Nos devuelve los 5 primeros registros del df

,Nombre,Apellido 1,Apellido 2,Sexo,Municipio,Provincia,DNI,NIF,Edad,ColorFavorito
0,Francisco,Castro,Cano,H,Lérida,Lérida,8805982,H,6.0,Verde
1,Xavier,Gómez,Rendón,H,Córdoba,Córdoba,26616576,X,77.0,Azul
2,Carmen,Vázquez,Trenado,M,Sabadell,Barcelona,47915145,NaN,NaN,Rojo
3,Anna,Alonso,López,M,NaN,Zaragoza,23362379,Z,24.0,Rojo
4,Manuel,López,Martínez,H,Palma de Mallorca,Islas Baleares,54999682,N,28.0,Amarillo


In [7]:
df.index # Nos da información del número de registros

RangeIndex(start=0, stop=100, step=1)

In [8]:
df.columns # Nos da información sobre las columnas

Index(['Nombre', 'Apellido 1', 'Apellido 2', 'Sexo', 'Municipio', 'Provincia',
       'DNI', 'NIF', 'Edad', 'ColorFavorito'],
      dtype='object')

## Filtrado con máscaras
Una de las tareas básicas cuando empezamos a procesar una base de datos, es filtrar los datos por diversos criterios. Para ello en pandas utilizamos máscaras. 

Hacer una máscara es tan sencillo como:

In [9]:
df_municipio_mask = df['Municipio'] == 'Valencia'

In [10]:
df_municipio_mask

0     False
1     False
2     False
3     False
4     False
5     False
6     False
7     False
8     False
9      True
10    False
11    False
12    False
13    False
14    False
15    False
16    False
17    False
18    False
19    False
20    False
21    False
22    False
23    False
24    False
25    False
26    False
27    False
28    False
29    False
      ...  
70    False
71    False
72    False
73    False
74    False
75    False
76    False
77    False
78    False
79    False
80    False
81    False
82    False
83     True
84    False
85    False
86    False
87    False
88    False
89    False
90    False
91     True
92    False
93    False
94    False
95     True
96    False
97    False
98    False
99    False
Name: Municipio, Length: 100, dtype: bool

In [11]:
df_municipio_mask.value_counts()

False    94
True      6
Name: Municipio, dtype: int64

Una máscara en el fondo es otro data frame del mismo tamaño que la misma columna que la original con valores booleanos.

In [12]:
df_municipio_mask.head()

0    False
1    False
2    False
3    False
4    False
Name: Municipio, dtype: bool

Filtrar con una máscara es tan fácil como utilizarla 

In [13]:
df[df_municipio_mask].head() # Todos los usuarios que viven en Valencia

,Nombre,Apellido 1,Apellido 2,Sexo,Municipio,Provincia,DNI,NIF,Edad,ColorFavorito
9,Alberto,González,Francés,H,Valencia,Valencia,2019499,F,23.0,Azul
43,Rubén,Ortiz,González,H,Valencia,Valencia,42769633,Y,80.0,NaN
61,María,Álvarez,Hernández,M,Valencia,Valencia,26406576,T,3.0,Rojo
83,Miguel,Hijano,Planas,H,Valencia,Valencia,96440536,H,53.0,Violeta
91,Daniel,Soria,Ordoñez,H,Valencia,Valencia,45282761,Q,37.0,Violeta


También podemos invertir el filtrado

In [14]:
df[~df_municipio_mask].head()

,Nombre,Apellido 1,Apellido 2,Sexo,Municipio,Provincia,DNI,NIF,Edad,ColorFavorito
0,Francisco,Castro,Cano,H,Lérida,Lérida,8805982,H,6.0,Verde
1,Xavier,Gómez,Rendón,H,Córdoba,Córdoba,26616576,X,77.0,Azul
2,Carmen,Vázquez,Trenado,M,Sabadell,Barcelona,47915145,NaN,NaN,Rojo
3,Anna,Alonso,López,M,NaN,Zaragoza,23362379,Z,24.0,Rojo
4,Manuel,López,Martínez,H,Palma de Mallorca,Islas Baleares,54999682,N,28.0,Amarillo


Y recordando cómo filtrar por valores nulos...

In [15]:
df_sexo_nula = df['Sexo'].isnull()
df[df_sexo_nula]

,Nombre,Apellido 1,Apellido 2,Sexo,Municipio,Provincia,DNI,NIF,Edad,ColorFavorito
5,María Dolores,Sánchez,Fernández,NaN,Alhaurín de la Torre,Málaga,11050962,Z,7.0,Verde
25,Carmen,Alvariño,Arias,NaN,Bilbao,Vizcaya,23055504,M,7.0,Rojo
35,Emilio,Vázquez,Gil Ortega,NaN,Barcelona,Barcelona,78666748,D,36.0,Azul
51,Iván,Gil,Diez,NaN,Barcelona,Barcelona,3424034,R,39.0,Violeta


In [16]:
df[df.isnull().any(axis=1)]

,Nombre,Apellido 1,Apellido 2,Sexo,Municipio,Provincia,DNI,NIF,Edad,ColorFavorito
2,Carmen,Vázquez,Trenado,M,Sabadell,Barcelona,47915145,NaN,NaN,Rojo
3,Anna,Alonso,López,M,NaN,Zaragoza,23362379,Z,24.0,Rojo
5,María Dolores,Sánchez,Fernández,NaN,Alhaurín de la Torre,Málaga,11050962,Z,7.0,Verde
25,Carmen,Alvariño,Arias,NaN,Bilbao,Vizcaya,23055504,M,7.0,Rojo
35,Emilio,Vázquez,Gil Ortega,NaN,Barcelona,Barcelona,78666748,D,36.0,Azul
43,Rubén,Ortiz,González,H,Valencia,Valencia,42769633,Y,80.0,NaN
51,Iván,Gil,Diez,NaN,Barcelona,Barcelona,3424034,R,39.0,Violeta


### Columnas que contienen textos

También podemos acceder a funciones de procesamiento de textos con la propiedad `str`. Se puede consultar en https://pandas.pydata.org/pandas-docs/stable/user_guide/text.html todas las opciones disponibles. 

Nosotros revisaremos en este worksheet una par de ellas muy frecuentes. Con `contains` podemos quedarnos con todas las cadenas que contienen otra dada.

In [17]:
df[df['Municipio'].str.contains('a', na=False)].head() # Municipios cuyo nombre contiene una 'a'

,Nombre,Apellido 1,Apellido 2,Sexo,Municipio,Provincia,DNI,NIF,Edad,ColorFavorito
0,Francisco,Castro,Cano,H,Lérida,Lérida,8805982,H,6.0,Verde
1,Xavier,Gómez,Rendón,H,Córdoba,Córdoba,26616576,X,77.0,Azul
2,Carmen,Vázquez,Trenado,M,Sabadell,Barcelona,47915145,NaN,NaN,Rojo
4,Manuel,López,Martínez,H,Palma de Mallorca,Islas Baleares,54999682,N,28.0,Amarillo
5,María Dolores,Sánchez,Fernández,NaN,Alhaurín de la Torre,Málaga,11050962,Z,7.0,Verde


Las máscaras se pueden aplicar de forma combinada utilizando operadores binarios. Por ejemplo para combinar (AND) dos máscaras a la vez:

In [18]:
df_mask_municipio_y_nombre = (df['Municipio'].isin(['Lérida', 'Sabadell']) & df['Nombre'].str.contains('C'))

In [19]:
df[df_mask_municipio_y_nombre]

,Nombre,Apellido 1,Apellido 2,Sexo,Municipio,Provincia,DNI,NIF,Edad,ColorFavorito
2,Carmen,Vázquez,Trenado,M,Sabadell,Barcelona,47915145,NaN,NaN,Rojo


## Aplicación de filtros
Probablemente una de las funciones más utilizadas en pandas en la función `apply` que nos permite ejecutar una función arbitraria a un dataframe.

Por ejemplo, si creamos un dataframe con los siguientes valores:

In [20]:
valores = {'X' : [2, 4, 5], 'Y' : [1, 3, 5]}
df2 = pd.DataFrame.from_dict(valores)
df2.head()

,X,Y
0,2,1
1,4,3
2,5,5


Podemos utilizar la función `apply` con la función `np.sqrt` para aplicarlo elemento a elemento:

In [21]:
df2.apply(np.sqrt)

,X,Y
0,1.414214,1.000000
1,2.000000,1.732051
2,2.236068,2.236068


También podemos utilizar nuestra propia función:

In [22]:
def eleva_al_cuadrado(valor):
    return valor*valor

df2.apply(eleva_al_cuadrado)

,X,Y
0,4,1
1,16,9
2,25,25


Además, con `apply`, podemos aplicar funciones a nivel de columna:

In [23]:
df2.apply(np.sum)

X    11
Y     9
dtype: int64

Si pasamos el parámetro `axis=0`, podemos aplicar a nivel de fila:

In [24]:
df2.apply(np.sum, axis=1)

0     3
1     7
2    10
dtype: int64

Aunque puede parecer muy potente e intuitivo, se aconseja evitar el comando `apply` siempre que sea posible y utilizar funciones nativas de pandas. Por ejemplo, para la suma:

In [25]:
df2.sum()

X    11
Y     9
dtype: int64

In [26]:
df2.sum(axis=1)

0     3
1     7
2    10
dtype: int64

De esta última manera la operación se vectoriza y termina mucho más rápido que `apply` que lo realiza de manera iterativa

In [27]:
%timeit df2.sum()

142 µs ± 9.6 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [28]:
%timeit df2.apply(np.sum)

757 µs ± 57.3 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


### Creación de columnas nuevas

La creación de columnas complementarias nos ayuda a entender los datos de una forma más resumida. También es importante tener esta técnica a la hora de trabajar con algoritmos, puesto que muchas veces si a un algoritmo le damos la información masticada podrá entender mejor el dataset. Esto es conocido como **feature engineering**.

En este caso, vamos a convertir una variable continua en una categórica, un caso clásico muy utilizado tanto para visualización como para preprocesamiento de datos previo al entrenamiento de un algoritmo.

In [29]:
def columna_edad_categorica(valor):
    if np.isnan(valor):
        return valor
    
    if valor <= 18:
        return 'J'
    elif valor > 18 and valor <= 65:
        return 'M'
    else:
        return 'V'

In [30]:
df['Edad_Categorica'] = df['Edad'].apply(columna_edad_categorica)
df['Edad_Categorica'].head()

0      J
1      V
2    NaN
3      M
4      M
Name: Edad_Categorica, dtype: object

## Generación de datos ficticios
Para generar fuentes de datos ficticias vamos a usar `faker` (https://faker.readthedocs.io/en/master/) que es un paquete python que nos permite generar aleatoriamente tipos de datos.

Esta paquete no viene por defecto en anaconda, pero podemos instalarlo fácilmente con el comando `conda install -c conda-forge faker`:

* En Linux lo ejecutamos directamente desde la terminal
* En Windows abriremos Anaconda Navigator -> Environments -> (seleccionamos el entorno que estemos usando) -> Simbolo de 'Play' -> Open Terminal

Una vez instalado podemos cargarlo de la siguiente manera:

In [31]:
from faker import Faker

Una vez cargado el paquete debemos instanciarlo, para ello debemos proveerle a faker el locale (idioma y población) con el que vamos a trabajar. En este caso vamos a trabajar con Español de España `es_ES`.

In [32]:
fake = Faker('es_ES')

Una vez instanciado faker, es muy sencilla su utilización. Simplemente sobre el objeto `fake` podemos aplicar varios commandos como:

In [33]:
fake.name()

'Luisa Roldán Pastor'

In [34]:
fake.address()

'Calle Nicolás Alcázar 102 Apt. 89 \nCeuta, 86118'

In [35]:
fake.text()

'Ratione inventore aut sunt dolorum. Quia iusto totam nisi voluptates aspernatur quas. Rerum eveniet porro quae.'

Adicionalmente a los métodos anteriores `faker` proporciona muchas más funciones, pero éstas se utilizan en base a `providers` que son contenedores que aglutinan grupos de funciones.

Para poder consultar la lista de proveedores disponibles en https://faker.readthedocs.io/en/latest/providers.html 

Nosotros vamos a probar el proveedor `faker.providers.internet` que nos permite generar `ips`, `emails` y otros tipos de datos interesantes en internet.

In [36]:
from faker.providers import internet

In [37]:
fake.email()

'jose-mariamalo@yahoo.com'

In [38]:
fake.free_email()

'carballomarc@yahoo.com'

In [39]:
fake.ipv4()

'207.128.90.251'

In [40]:
fake.url()

'http://bueno.info/'

## Anonimización

Anonimizar consiste en alterar un conjunto de datos de forma sea imposible identificar cada individuo del dataset, pero que las propiedades poblacionales del mismo se mantengan.

Por ejemplo, vamos a anonimizar el dataset `df` para ello lo primero tenemos que identificar que campos o combinación de campos resultan sensibles:

In [41]:
df.head()

,Nombre,Apellido 1,Apellido 2,Sexo,Municipio,Provincia,DNI,NIF,Edad,ColorFavorito,Edad_Categorica
0,Francisco,Castro,Cano,H,Lérida,Lérida,8805982,H,6.0,Verde,J
1,Xavier,Gómez,Rendón,H,Córdoba,Córdoba,26616576,X,77.0,Azul,V
2,Carmen,Vázquez,Trenado,M,Sabadell,Barcelona,47915145,NaN,NaN,Rojo,NaN
3,Anna,Alonso,López,M,NaN,Zaragoza,23362379,Z,24.0,Rojo,M
4,Manuel,López,Martínez,H,Palma de Mallorca,Islas Baleares,54999682,N,28.0,Amarillo,M


Como podemos ver, hay algunos campos que resultan claramente identificativos:
* Nombre y apellidos de forma conjunta 
* DNI + NIF

Para anonimizar el nombre y apellidos utilizaremos `faker`:

In [42]:
def anon_name(row):
    if row['Sexo'] == 'H':
        row['Nombre'] = fake.first_name_male()
    elif row['Sexo'] == 'M':
        row['Nombre'] = fake.first_name_female()
    
    row['Apellido 1'] = fake.last_name()
    row['Apellido 2'] = fake.last_name()
    return row

df = df.apply(anon_name, axis=1)

In [43]:
df.head()

,Nombre,Apellido 1,Apellido 2,Sexo,Municipio,Provincia,DNI,NIF,Edad,ColorFavorito,Edad_Categorica
0,Cesar,Hoyos,Tapia,H,Lérida,Lérida,8805982,H,6.0,Verde,J
1,Hector,Nuñez,Manrique,H,Córdoba,Córdoba,26616576,X,77.0,Azul,V
2,Amparo,Antón,Martí,M,Sabadell,Barcelona,47915145,NaN,NaN,Rojo,NaN
3,Rosa,Viana,Larrañaga,M,NaN,Zaragoza,23362379,Z,24.0,Rojo,M
4,Jose Ignacio,Pelayo,Reina,H,Palma de Mallorca,Islas Baleares,54999682,N,28.0,Amarillo,M


Y haremos lo propio con los DNIs. En este caso es un poco más complicado, porque

In [44]:
def anon_dni(row):
    row['DNI'] = fake.numerify(text="########")
    return row

df = df.apply(anon_dni, axis=1)
df.head()

,Nombre,Apellido 1,Apellido 2,Sexo,Municipio,Provincia,DNI,NIF,Edad,ColorFavorito,Edad_Categorica
0,Cesar,Hoyos,Tapia,H,Lérida,Lérida,38201193,H,6.0,Verde,J
1,Hector,Nuñez,Manrique,H,Córdoba,Córdoba,88867605,X,77.0,Azul,V
2,Amparo,Antón,Martí,M,Sabadell,Barcelona,38301221,NaN,NaN,Rojo,NaN
3,Rosa,Viana,Larrañaga,M,NaN,Zaragoza,54231766,Z,24.0,Rojo,M
4,Jose Ignacio,Pelayo,Reina,H,Palma de Mallorca,Islas Baleares,59538561,N,28.0,Amarillo,M


### Barajado de datos

Una estrategia habitual en la anonimización es el barajado de datos que consiste en desordenar los valores de las columnas. Si el dataset es suficientemente grande y la columna no es identificativa individualmente, esta técnica puede dar buenos resultados.

Para barajar los datos lo podemos hacer con el método 'np.random.shuffle'. Es importante destacar que este método, realiza modificaciones `in place` sobre los datos. Por lo que tendremos que convertir a lista nuestros datos de forma intermedia.

In [45]:
def baraja_columna(df, nombre_columna):
    temp = list(df[nombre_columna])
    np.random.shuffle(temp)
    df[nombre_columna] = temp
    return df

In [46]:
df['ColorFavorito'].head()

0       Verde
1        Azul
2        Rojo
3        Rojo
4    Amarillo
Name: ColorFavorito, dtype: object

In [47]:
df = baraja_columna(df, 'ColorFavorito')

In [48]:
df['ColorFavorito'].head()

0      Verde
1      Verde
2      Verde
3    Fucshia
4       Rojo
Name: ColorFavorito, dtype: object

## Quizz

* ¿Es suficiente hacer una conversión de caracteres para anonimizar?
* ¿Es posible anonimizar cualquier tipo de datos?
* Se te ocurre algún criterio, para garantizar que un dataset es "anónimo".
* Lectura opcional: https://www.wired.com/2007/12/why-anonymous-data-sometimes-isnt/